In [47]:
import matplotlib.pyplot as plt
import math
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import folium
from collections import Counter
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import getName
from ipynb.fs.defs.functions import translate_name
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from tqdm.notebook import tqdm as tqdm

df = pd.read_csv("../request_cr.csv", delimiter=",", encoding = "ISO-8859-1")
dft = pd.read_csv("../distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

def manage_exception(string):
    if(string.lower().strip() == 'piadena drizzona'):
        return 'Piadena, Platina, 40'
    elif(string.lower().strip() == 'brescia'):
        return 'manerbio, insurrezione nazionale'
    elif(string.lower().strip() == 'centro fumetto a. pazienza' or string.lower().strip() == 'cremona. la piccola biblioteca' or string.lower().strip() == 'uff. bib. cremona'):
        return 'cremona, ugolani dati, 4'
    elif(string.lower().strip() == 'casale cremasco vidolasco'):
        return 'Casale Cremasco-Vidolasco, Roma, 56'
    elif(string.lower().strip() == 'cremona-m.i.t.o.'):
        return 'Cremona, Palosca n. 2'
    elif(string.lower().strip() == 'cremona - manin (scolastica)'):
        return 'Cremona, Felice Cavallotti, 2'
    elif(string.lower().strip() == 'gadesco pieve delmona'):
        return 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
    elif(string.lower().strip() == 'cremona - cda'):
        return 'cremona, sesto 39'
    elif(string.lower().strip() == 'crema'):
        return 'Crema, Civerchi, 9'
    elif(string.lower().strip() == 'martignana di po, libertà, 64'):
        return 'martignana di po, libertà, 64'
    else:
        return string

In [51]:
#frequenza degli scambi tra i e j con richiesta nel giorno g
def e (g, i, j, k):
    v =[i, j, g]
    return len([index for index, value in enumerate(k[g]) if value == v])
#e('martedi', 'Offanengo, Clavelli Martini, 5', 'Soncino, Dante Alighieri, 23')

def dist(s, e):
#     if s == e:
#         return 7
    if s < e:
        return e-s
    if s >= e:
        return 7-(s-e)

def f(i, g, j, h, fur1, fur2, k):
    w = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi", "sabato", "domenica"]
    g = week.index(g)
    s = 0
    f1_occurrences = getOccurenceDays(fur1, fur2, i)
    
    try:
        q = list([x for x in f1_occurrences if x>=g] + [x for x in f1_occurrences if x<g])[-1]
    except:
        print(i, g, j, h, sep="    ")
        
    if(q<g):
        for m in range(q, g):
            s += dist(m, w.index(h)) * e(m, i, j, k)
    elif(q>=g):
        for m in list(range(q, 7)) + list(range(0, g)):
            s += dist(m, w.index(h)) * e(m, i, j, k)
    return s       

In [48]:
#Take schedulation and return a list that contains the day(in index) where the city appears
def getOccurenceDays(fur1, fur2, city, dframe, b):
    week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    l = []
    for d in fur1.columns:
        if b == 1:
            cur_route1 = fur1[d].dropna()[1:len(fur1[d].dropna())-1]
        else:
            cur_route1 = fur1[d].dropna()
            
        if getName(dframe, city) in list(cur_route1):
            l.append(week.index(d))
            
    for d in fur2.columns:
        if b == 1:
            cur_route2 = fur2[d].dropna()[1:len(fur2[d].dropna())-1]
        else:
            cur_route2 = fur2[d].dropna()
            
        if getName(dframe, city) in list(cur_route2):
            l.append(week.index(d))
          
    return list(set(l))

def getDistance(s, e):
#     if s == e:
#         return 7
    if s < e:
        return e-s
    if s > e:
        return 7-(s-e)

def mean_wdays(fur1, fur2, city_s, city_e, dframe, b):
    s_list = getOccurenceDays(fur1, fur2, city_s, dframe, b)
    e_list = getOccurenceDays(fur1, fur2, city_e, dframe, b)
    results = []
    for s in s_list:
        for e in e_list:
            results.append(getDistance(s, e))
    
    if not results:
        #print(s_list, city_s, sep = ' $ ')
        #print(e_list, city_e, sep = ' $ ')
        return -1
    return round(sum(results) / len(results), 1)

def compute_mean_wdays(dframe, fur1, fur2, b):
    week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    
    f1 = translate_name(dframe, fur1)
    f2 = translate_name(dframe, fur2)
    
    indx = [getName(dframe, x) for x in dframe.columns]
    waiting_days = pd.DataFrame({}, index = indx, columns = indx)
    
    for s in indx:
        for e in indx:
            waiting_days.loc[s, e] = mean_wdays(f1, f2, s, e, dframe, b)
            
    return waiting_days

def get_carla(fur1, fur2, city_s, city_e, dframe, b):
    s_list = getOccurenceDays(fur1, fur2, city_s, dframe, b)
    e_list = getOccurenceDays(fur1, fur2, city_e, dframe, b)
    results = []
    dists = []
    ocs = []
    for s in s_list:
        for e in e_list:
            results.append((s, e))
            dists.append(getDistance(s, e))
    
    if not results:
        #print(s_list, city_s, sep = ' $ ')
        #print(e_list, city_e, sep = ' $ ')
        return -1
    return results, dists

indx = [getName(dft, x) for x in dft.columns]

In [ ]:
compute_mean_wdays(dft, fur1_MVLS, fur2_MVLS,1 ).to_csv("../Dati/distanze/waiting_days_MVLS.csv", sep=";", encoding = "ISO-8859-1")
compute_mean_wdays(dft, fur1_2opt, fur2_2opt, 1).to_csv("../Dati/distanze/waiting_days_2opt.csv", sep=";", encoding = "ISO-8859-1")
compute_mean_wdays(dft, fur1_greedy, fur2_greedy, 0).to_csv("../Dati/distanze/waiting_days_greedy.csv", sep=";", encoding = "ISO-8859-1")

waiting_days_MVLS = pd.read_csv("../Dati/distanze/waiting_days_MVLS.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
waiting_days_2opt = pd.read_csv("../Dati/distanze/waiting_days_2opt.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
waiting_days_greedy = pd.read_csv("../Dati/distanze/waiting_days_greedy.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
orders = pd.read_csv("../Dati/distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
indx = [getName(dft, x) for x in dft.columns]
weighted_index_MVLS = pd.DataFrame({}, index = indx, columns = indx)
weighted_index_2opt = pd.DataFrame({}, index = indx, columns = indx)
weighted_index_greedy = pd.DataFrame({}, index = indx, columns = indx)

for s in indx:
    for e in indx:
        weighted_index_MVLS.loc[s, e] = waiting_days_MVLS.loc[s, e] * orders.loc[s, e]
        weighted_index_2opt.loc[s, e] = waiting_days_2opt.loc[s, e] * orders.loc[s, e]
        weighted_index_greedy.loc[s, e] = waiting_days_greedy.loc[s, e] * orders.loc[s, e]

In [ ]:
weighted_index_MVLS.to_csv("../Dati/distanze/weighted_wmean_MVLS.csv", sep=";", encoding = "ISO-8859-1")
weighted_index_2opt.to_csv("../Dati/distanze/weighted_wmean_2opt.csv", sep=";", encoding = "ISO-8859-1")
weighted_index_greedy.to_csv("../Dati/distanze/weighted_wmean_greedy.csv", sep=";", encoding = "ISO-8859-1")

In [50]:
dat = pd.DataFrame({}, columns = ['start', 'start_day', 'destination', 'destination_day', 'product'])
orders = pd.read_csv("../distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1_2opt = pd.read_csv("../turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur2_2opt = pd.read_csv("../turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
f1 = translate_name(dft, fur1_2opt)
f2 = translate_name(dft, fur2_2opt)

In [ ]:
#OSS
dat = pd.DataFrame({}, columns = ['start', 'start_day', 'destination', 'destination_day', 'product'])
orders = pd.read_csv("../distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1_greedy = pd.read_csv("../turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur2_greedy = pd.read_csv("../turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
f1 = translate_name(dft, fur1_greedy)
f2 = translate_name(dft, fur2_greedy)
k = indx
k.pop(15)
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
indx = [getName(dft, x) for x in dft.columns]
indx.pop(15)
for i in tqdm(range(len(k))):
    f1_occ = getOccurenceDays(f1, f2, indx[i], dft, 1)
    for d1 in week:
        if week.index(d1) in f1_occ:
            for j in k:
                f2_occ = getOccurenceDays(f1, f2, j, dft, 1)
                for d2 in week:
                    if indx[i] == j:
                        dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(j), 'destination_day': week.index(d2), 
                                          'product': 0}, ignore_index = True)
                        continue
                        
                    if week.index(d2) in f2_occ:
                        l, dists = get_carla(f1, f2, indx[i], j, dft, 1)
                        dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(j), 'destination_day': week.index(d2), 
                                          'product': round(getDistance(week.index(d1), week.index(d2))*orders.loc[indx[i], j], 
                                                           1)}, ignore_index = True)
                    else:
                        dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(j), 'destination_day': week.index(d2), 
                                          'product': 0}, ignore_index = True)
        else:
            for w in k:
                for d3 in week:
                    dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(w), 'destination_day': week.index(d3), 
                                      'product': 0}, ignore_index = True)

In [ ]:
#FOSS
fur1 = pd.read_csv("../turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]

x = list(dft.columns)
x.pop(15)
results = []
for i in tqdm(range(len(x))):
    for g in week:
        for j in x:
            if x[i] == j:
                for h in week:
                    results.append(str(i) + " " + str(week.index(g)) + " " + str(x.index(j)) + " " + str(week.index(h)) + " " + str(0))
                continue
            
            tmpx = x[i]
#             if x[i] == 'Crema, Dante Alighieri, 49':
#                 tmpx = 'Crema, Civerchi, 9'
            
            tmpj = j
#             if j == 'Crema, Dante Alighieri, 49':
#                 tmpj = 'Crema, Civerchi, 9'
                
            for h in week:
                results.append(str(i) + " " + str(week.index(g)) + " " + str(x.index(j)) + " " + str(week.index(h)) + " " + str(f(tmpx, g, tmpj, h, fur1, fur2, k)))

In [ ]:
#pd.set_option('display.float_format', '{:.2f}'.format)
dat.to_csv("../turni_f/optimized_start/datoss.dat", sep=" ", encoding = "ISO-8859-1", index = False, header = False)
dfprova = pd.read_csv("../turni_f/optimized_start/datoss.dat", delimiter=" ", encoding = "ISO-8859-1", names = ['x', 'y', 'z', 'w', 'v'])
len(dfprova)

In [34]:
def getOccurenceNumber(fur1, fur2, city):
    week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    l = []
    for d in fur1.columns:
        cur_route1 = fur1[d].dropna()[1:len(fur1[d].dropna())-1]
            
        l += [x for x in cur_route1 if x==city]
    
    for d in fur2.columns:
        cur_route2 = fur2[d].dropna()[1:len(fur2[d].dropna())-1]   
        l += [x for x in cur_route2 if x==city]     
    return len(l)

In [39]:
#param v generator
fur1 = pd.read_csv("../turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]

x = list(dft.columns)
x.pop(15)
results = []
n = 0
for city in x:
    results.append(str(x.index(city)) + " " + str(getOccurenceNumber(fur1, fur2, city)))
    n+=getOccurenceNumber(fur1, fur2, city)

In [44]:
pd.DataFrame(results).to_csv("paramv.txt", header=False, index=False, sep="\n")